# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

### This notebook shows examples of using Glue to explore and handle iceberg data
To follow this example notebook, execute the cells in order.  
The keyboard shortcut to execute the current cell and jump to the following is: Shift+Enter.

####  Run this cell to set up and start your interactive session.


In [7]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2
%spark_conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://es-datalake/silver/iceberg-example/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.defaultCatalog=glue_catalog
%%configure
{
   "--datalake-formats": "iceberg"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
Previous Spark configuration: None
Setting new Spark configuration to: spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://es-datalake/silver/iceberg-example/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCata

In [9]:
%spark_conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://es-datalake/silver/iceberg-example/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.defaultCatalog=glue_catalog

Previous Spark configuration: spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://es-datalake/silver/iceberg-example/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.defaultCatalog=glue_catalog
Setting new Spark configuration to: spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://es-datalake/silver/iceberg-example/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.defaultCatalog=glue_cata

In [1]:
catalog_name = "glue_catalog"
bucket_name = "es-datalake"
bucket_prefix = "silver/iceberg-example2/"
database_name = "iceberg_sql2"
table_name = "medicare"
warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 2880
Session ID: 8c9409a6-e5ce-4018-a5cc-b9823d07d93f
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
--datalake-formats iceberg
--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://es-datalake/silver/iceberg-example/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.defaultCatalog=glue_catalog
Waiting for session 8c9409a6-e5ce-4018-a5cc-b9823d07d93f to get into ready status...
Session 8c9409a6-e5ce-4018-a5cc-b9823d07d93f has been created.



In [2]:
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [3]:
query = f"""
CREATE DATABASE IF NOT EXISTS {database_name}
"""
spark.sql(query)

DataFrame[]


In [4]:
# If you data doesn't have a table in the catalog, you can use a temporary view to use SQL
# Here we read all the CSV files under the indicated s3 path
medicareDF = spark.read.csv("s3://awsglue-datasets/examples/medicare/", header=True)
# If there data has a reasonable size (like in this case), we can cache in memory/disk (depending on cluster size) 
# so after the first query, the following no longer have to go to read and parse from s3
medicareDF.cache()
# Instead of using the DataFrame API, you can register it as a view for SQL usage like this:
medicareDF.registerTempTable("tmp_medicare")

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.


In [5]:
query = f"""
CREATE TABLE {catalog_name}.{database_name}.{table_name}
USING iceberg
LOCATION '{warehouse_path}'
TBLPROPERTIES ("format-version"="2")
AS SELECT * FROM tmp_medicare
"""
spark.sql(query)

DataFrame[]


In [6]:
%%sql
SELECT * FROM glue_catalog.iceberg_sql2.medicare.history

+--------------------+-------------------+---------+-------------------+
|     made_current_at|        snapshot_id|parent_id|is_current_ancestor|
+--------------------+-------------------+---------+-------------------+
|2025-04-15 03:53:...|6277996127910655019|     null|               true|
+--------------------+-------------------+---------+-------------------+


In [6]:
query = f"""
INSERT INTO {catalog_name}.{database_name}.{table_name}
(SELECT * FROM tmp_{table_name})
"""
spark.sql(query)

DataFrame[]


In [8]:
%%sql
SELECT * FROM glue_catalog.iceberg_sql2.medicare.history

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2025-04-15 03:53:...|6277996127910655019|               null|               true|
|2025-04-15 03:56:...|5173792975111062545|6277996127910655019|               true|
+--------------------+-------------------+-------------------+-------------------+


In [9]:
%%sql
SELECT count(*) FROM glue_catalog.iceberg_sql2.medicare

+--------+
|count(1)|
+--------+
|  326130|
+--------+


In [10]:
%%sql
SELECT count(*) FROM glue_catalog.iceberg_sql2.medicare for version as of 6277996127910655019

+--------+
|count(1)|
+--------+
|  163065|
+--------+


### Queries in Athena
The same queries in the Athena SQL format are
```
select count(*) from medicare;

select * from "medicare$history";

select count(*) from medicare for version as of 6277996127910655019;
```


In [ ]:
# Queries in Athena SQL format

# select count(*) from medicare;

# select * from "medicare$history";

# select count(*) from medicare for version as of 6277996127910655019;